<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
</center>
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>Практика. Где логистическая регрессия хороша и где не очень

### Анализ отзывов IMDB к фильмам

Будем решать задачу бинарной классификации отзывов IMDB к фильмам. Имеется обучающая выборка с размеченными отзывами, по 12500 отзывов известно, что они хорошие, еще про 12500 – что они плохие. Здесь уже не так просто сразу приступить к машинному обучению, потому что готовой матрицы $X$ нет  – ее надо приготовить. Будем использовать самый простой подход – мешок слов ("Bag of words"). При таком подходе признаками отзыва будут индикаторы наличия в нем каждого слова из всего корпуса, где корпус – это множество всех отзывов. Идея иллюстрируется картинкой

<img src="img/bag_of_words.svg" width=80%>

In [ ]:
#Тмпорт библиотек
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

**Загрузим данные [отсюда](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz) (это прямая ссылка на скачивание, а [вот](http://ai.stanford.edu/~amaas/data/sentiment/) описание набора данных). В обучающей и тестовой выборках по 12500 тысяч хороших и плохих отзывов к фильмам.**

In [ ]:
# поменяйте путь к файлу
reviews_train = load_files("/files/datasets/mlcourse/04/aclImdb/train")
# загружаем данные
text_train, y_train = reviews_train.data, reviews_train.target

In [ ]:
print("Number of documents in training data: %d" % len(text_train))
print(np.bincount(y_train))

In [ ]:
# поменяйте путь к файлу
reviews_test = load_files("/files/datasets/mlcourse/04/aclImdb/train")
text_test, y_test = # ваш код здесь
print("Number of documents in test data: %d" % len(text_test))
print(np.bincount(y_test))

**Пример отзыва и соответствующей метки.**

In [ ]:
print(text_train[1])

In [ ]:
y_train[1] # плохой отзыв

In [ ]:
print(text_train[2]) # взгляните на отзыв

In [ ]:
y_train[2] # какой это отзыв?

## Простой подсчет слов

**Составим словарь всех слов с помощью CountVectorizer.**

In [ ]:
cv = CountVectorizer()
cv.fit(text_train)

len(cv.vocabulary_)

**Посмотрим на примеры полученных "слов" (лучше их называть токенами). Видим, что многие важные этапы обработки текста мы тут пропустили.**

In [ ]:
print(cv.get_feature_names()[:50])
print(cv.get_feature_names()[50000:50050])

**Закодируем предложения из текстов обучающей выборки индексами входящих слов. Используем разреженный формат.**

In [ ]:
X_train = cv.transform(text_train)
X_train

**Посмотрим, как преобразование подействовало на одно из предложений.**

In [ ]:
print(text_train[19726])

In [ ]:
X_train[19726].nonzero()[1]

In [ ]:
X_train[19726].nonzero()

**Преобразуем так же тестовую выборку.**

In [ ]:
X_test = # ваш код здесь

**Обучим логистическую регрессию.**

In [ ]:
%%time
logit = LogisticRegression(n_jobs=-1, random_state=7)
logit.fit(X_train, y_train)

**Посмотрим на доли правильных ответов на обучающей и тестовой выборках.**

In [ ]:
round(logit.score(X_train, y_train), 3), round(logit.score(X_test, y_test), 3),

**Коэффициенты модели можно красиво отобразить.**

In [ ]:
def visualize_coefficients(classifier, feature_names, n_top_features=25):
    # get coefficients with large absolute values 
    coef = classifier.coef_.ravel()
    positive_coefficients = np.argsort(coef)[-n_top_features:]
    negative_coefficients = np.argsort(coef)[:n_top_features]
    interesting_coefficients = np.hstack([negative_coefficients, positive_coefficients])
    # plot them
    plt.figure(figsize=(15, 5))
    colors = ["red" if c < 0 else "blue" for c in coef[interesting_coefficients]]
    plt.bar(np.arange(2 * n_top_features), coef[interesting_coefficients], color=colors)
    feature_names = np.array(feature_names)
    plt.xticks(np.arange(1, 1 + 2 * n_top_features), feature_names[interesting_coefficients], rotation=60, ha="right");


In [ ]:
def plot_grid_scores(grid, param_name):
    plt.plot(grid.param_grid[param_name], grid.cv_results_['mean_train_score'],
        color='green', label='train')
    plt.plot(grid.param_grid[param_name], grid.cv_results_['mean_test_score'],
        color='red', label='test')
    plt.legend();
    

In [ ]:
visualize_coefficients(logit, cv.get_feature_names())

**Подберем коэффициент регуляризации для логистической регрессии. Используем `sklearn.pipeline`, поскольку `CountVectorizer` правильно применять только на тех данных, на которых в текущий момент обучается модель (чтоб не "подсматривать" в тестовую выборку и не считать по ней частоты вхождения слов). В данном случае `pipeline` задает последовательность действий: применить `CountVectorizer`, затем обучить логистическую регрессию.**

In [ ]:
%%time
from sklearn.pipeline import make_pipeline

text_pipe_logit = make_pipeline(CountVectorizer(), 
                                LogisticRegression(n_jobs=-1, random_state=7))

text_pipe_logit.fit(text_train, y_train)
print(text_pipe_logit.score(text_test, y_test))

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

param_grid_logit = {'logisticregression__C': np.logspace(-5, 0, 6)}
grid_logit = GridSearchCV(text_pipe_logit, param_grid_logit, cv=3, n_jobs=-1)

grid_logit.fit(text_train, y_train)

**Лучшее значение C и соответствующее качество:**

In [ ]:
grid_logit.best_params_, grid_logit.best_score_

In [ ]:
plot_grid_scores(grid_logit, 'logisticregression__C')

На валидационной выборке:

In [ ]:
grid_logit.score(text_test, y_test)

**Теперь то же самое, но со случайным лесом. Видим, что с логистической регрессией мы достигаем большей доли правильных ответов меньшими усилиями**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=17)

In [ ]:
%%time
forest.fit(X_train, y_train)

In [ ]:
round(forest.score(X_test, y_test), 3)